In [114]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Dense,Dropout

In [108]:
data_train = pd.read_csv("data/train.csv")
data_test = pd.read_csv('data/test.csv')
data_target= pd.read_csv('data/gender_submission.csv')
data_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [110]:
data_train.replace(["female", "male"], [0, 1], inplace=True)
data_train.fillna(0, inplace=True)
X_train = np.array(data_train[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]])
Y_train = np.array(data_train[["Survived"]])
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
print(X_train.shape,Y_train.shape)


(891, 6) (891, 1)


<ipython-input-110-4039996578>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_train.replace(["female", "male"], [0, 1], inplace=True)


In [112]:
data_test.replace(["female", "male"], [0, 1], inplace=True)
data_test.fillna(0, inplace=True)
X_test = np.array(data_test[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare"]])
Y_test = np.array(data_target[["Survived"]])
scaler = StandardScaler()
X_test = scaler.fit_transform(X_test)
print(X_test.shape,Y_test.shape)


(418, 6) (418, 1)


In [115]:
train_data = tf.data.Dataset.from_tensor_slices((X_train, Y_train)).shuffle(10).batch(6)
test_data = tf.data.Dataset.from_tensor_slices((X_test, Y_test)).batch(6)

In [123]:
class Model(tf.keras.Model):
  def __init__(self):
    super().__init__()

    self.dense1 = Dense(6,activation="relu")
    self.dense2 = Dense(64,activation="relu")
    self.dropout1 = Dropout(0.3)
    self.dense3 = Dense(32,activation="relu")
    self.dense4 = Dense(2,activation="softmax" )

  def call(self,x):
     x = self.dense1(x)
     x = self.dense2(x)
     x = self.dropout1(x)
     x = self.dense3(x)
     x = self.dense4(x)

     return x

model = Model()

In [124]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()

In [125]:
train_loss = tf.keras.metrics.Mean()
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

test_loss = tf.keras.metrics.Mean()
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()

In [126]:
@tf.function
def train(imagse,labels):
    with tf.GradientTape() as tape:
        pred = model(imagse)
        loss = loss_function(labels,pred)
    gradients = tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels,pred)

In [127]:
@tf.function
def test(imagse,labels):
        pred = model(imagse)
        loss = loss_function(labels,pred)
        test_loss(loss)
        test_accuracy(labels,pred)

In [128]:
epochs = 50
for epoch in range(epochs):
  train_loss.reset_state()
  train_accuracy.reset_state()
  test_loss.reset_state()
  test_accuracy.reset_state()

  for imagse , labels in train_data:
    train(imagse,labels)

  for imagse , labels in test_data:
    test(imagse,labels)

  print("epoch :" ,epoch + 1,
        f"Train Loss :{train_loss.result()}",
        f"Train accuracy:{train_accuracy.result()}",
        f"Test Loss :   {test_loss.result()}",
        f"Test accuracy:{test_accuracy.result()}")


epoch : 1 Train Loss :0.5662529468536377 Train accuracy:0.7216610312461853 Test Loss :   0.4308476746082306 Test accuracy:0.8014354109764099
epoch : 2 Train Loss :0.4732266664505005 Train accuracy:0.7878788113594055 Test Loss :   0.35326722264289856 Test accuracy:0.8779904246330261
epoch : 3 Train Loss :0.44880178570747375 Train accuracy:0.8024691343307495 Test Loss :   0.32868996262550354 Test accuracy:0.8899521827697754
epoch : 4 Train Loss :0.43495967984199524 Train accuracy:0.8069584965705872 Test Loss :   0.3308371305465698 Test accuracy:0.8779904246330261
epoch : 5 Train Loss :0.4245304763317108 Train accuracy:0.8181818127632141 Test Loss :   0.3140662908554077 Test accuracy:0.8851674795150757
epoch : 6 Train Loss :0.42133432626724243 Train accuracy:0.8260381817817688 Test Loss :   0.31545940041542053 Test accuracy:0.8827751278877258
epoch : 7 Train Loss :0.4155079126358032 Train accuracy:0.8271604776382446 Test Loss :   0.3136958181858063 Test accuracy:0.8779904246330261
epoch :

In [129]:
jack = np.array([[3, 1, 20, 1, 1, 5]])
jack = scaler.transform(jack)


rose = np.array([[1, 0, 18, 2, 0, 100]])
rose = scaler.transform(rose)

jack_survival = model(jack)
rose_survival = model(rose)

print(f"Jack's survival probability: {jack_survival[0][1]:.2f}")
print(f"Rose's survival probability: {rose_survival[0][1]:.2f}")

Jack's survival probability: 0.09
Rose's survival probability: 0.97


In [ ]:
model.export("my_model3")
